In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [56]:
url_inicial = 'https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-numeros-anteriores-anio-2017'
url_root = 'https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-numeros-anteriores-anio-2017'
r=requests.get(url_inicial)

In [57]:
soup = BeautifulSoup(r.text, 'html.parser')

In [58]:
box = soup.find('div', class_='sumarios decada-2017')
volumen=box.findAll('div', class_='sumario row')
vol = [x.find('a').get('href')for x in volumen]
vol=[urljoin(url_root,i) for i in vol]
vol

['https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-sumario-vol-31-num-4-S0187533718X0003X',
 'https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-sumario-vol-31-num-3-S0187533718X00028',
 'https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-sumario-vol-31-num-2-S0187533717X00058',
 'https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-sumario-vol-31-num-1-S0187533717X00046']

In [59]:
vol2 =[]
for i in vol: 
    url_inicial1=i 
    r1 = requests.get(url_inicial1)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    box1 = soup1.find('div', class_='items') 
    volumen1=box1.findAll('div', class_='item')
    vol1 = [x.find('a').get('href')for x in volumen1]
    vol1 = [urljoin (url_root, i) for i in vol1]
    vol2 = [urljoin (url_root, i) for i in vol2]
    vol2+=vol1
    vol2+=vol1

In [60]:
vol2

['https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-articulo-la-investigacion-clinica-neonatologia-el-S0187533718300402',
 'https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-articulo-morbilidad-mortalidad-neonatos-1500g-S0187533718300311',
 'https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-articulo-contacto-piel-piel-al-nacimiento-S0187533718300335',
 'https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-articulo-factores-riesgo-asociados-el-desarrollo-S0187533718300323',
 'https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-articulo-resultados-maternos-neonatales-trillizos-nacidos-S018753371830013X',
 'https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-articulo-estado-vacunacion-prematuros-menores-1500g-S0187533718300116',
 'https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-articulo-experiencia-madres-durante-el-contacto-S018753371

In [61]:
def get_url_items(sopa,url):
    box1 = soup1.find('div', class_='items') 
    volumen1=box1.findAll('div', class_='item')
    vol1 = [x.find('a').get('href')for x in volumen1]
    return vol2

In [62]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break

Estoy en la pagina https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-numeros-anteriores-anio-2017


In [63]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

74

In [64]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [65]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Perinatología y Reproducción Humana'
    a='III. Medicina y Ciencias de la Salud'
    tem='Perinatología, Neonatología, Obstetricia, Reproducción Humana'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo Revista']=tire
    except AttributeError:
        content_book['Titulo Revista']=None
    #area
    try:
        area=a
        content_book['Area']=a
    except AttributeError:
        content_book['Area']=None
    #tematica
    try:
        tema=tem
        content_book['Tematica']=tem
    except AttributeError:
        content_book['Tematica']=None
    #titulo articulo
    try:
        titu=s_item.find('div', class_='elsevierItemTitulo').get_text(strip=True)
        content_book['Titulo Articulo']=titu
    except AttributeError:
        content_book['Titulo Articulo']=None
    #resumen
    try:
        resu=s_item.find('span', class_='elsevierStyleSections').get_text(strip=True)
        content_book['Resumen']=resu.replace("Objetivo","")
    except AttributeError:
        content_book['Resumen']=None
    #abstract
    try:
        abst=s_item.find('div', class_='logo').get_text(strip=True)
        content_book['Abstract']=abst
    except AttributeError:
        content_book['Abstract']=None
    #Link incial
    try:
        linkart=url
        content_book['Link Articulo']=linkart
    except AttributeError:
        content_book['Link Articulo']=None
    #link articulo
    try:
        link=s_item.find('div', class_="item-anchors desktop").find('a').get('href')
        content_book['Link PDF']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link PDF']=None
    return content_book

In [66]:
list_scraper=list_scraper[0:74]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3
estas escrapeando la pag 4
estas escrapeando la pag 5
estas escrapeando la pag 6
estas escrapeando la pag 7
estas escrapeando la pag 8
estas escrapeando la pag 9
estas escrapeando la pag 10
estas escrapeando la pag 11
estas escrapeando la pag 12
estas escrapeando la pag 13
estas escrapeando la pag 14
estas escrapeando la pag 15
estas escrapeando la pag 16
estas escrapeando la pag 17
estas escrapeando la pag 18
estas escrapeando la pag 19
estas escrapeando la pag 20
estas escrapeando la pag 21
estas escrapeando la pag 22
estas escrapeando la pag 23
estas escrapeando la pag 24
estas escrapeando la pag 25
estas escrapeando la pag 26
estas escrapeando la pag 27
estas escrapeando la pag 28
estas escrapeando la pag 29
estas escrapeando la pag 30
estas escrapeando la pag 31
estas escrapeando la pag 32
estas escrapeando la pag 33
estas escrapeando la pag 34
estas escrapeando la pag 35
es

In [67]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo Revista,Area,Tematica,Titulo Articulo,Resumen,Abstract,Link Articulo,Link PDF
0,Perinatología y Reproducción Humana,III. Medicina y Ciencias de la Salud,"Perinatología, Neonatología, Obstetricia, Repr...",La investigación clínica en neonatología en el...,«El tiempo destruye rápidamente los trabajos c...,,https://www.elsevier.es/es-revista-perinatolog...,https://www.elsevier.es/es-revista-perinatolog...
1,Perinatología y Reproducción Humana,III. Medicina y Ciencias de la Salud,"Perinatología, Neonatología, Obstetricia, Repr...","Morbilidad y mortalidad de neonatos < 1,500g i...",IntroducciónCon los avances en la atención per...,,https://www.elsevier.es/es-revista-perinatolog...,https://www.elsevier.es/es-revista-perinatolog...
2,Perinatología y Reproducción Humana,III. Medicina y Ciencias de la Salud,"Perinatología, Neonatología, Obstetricia, Repr...",Contacto piel a piel al nacimiento,IntroducciónDespués del nacimiento debemos fav...,,https://www.elsevier.es/es-revista-perinatolog...,https://www.elsevier.es/es-revista-perinatolog...
3,Perinatología y Reproducción Humana,III. Medicina y Ciencias de la Salud,"Perinatología, Neonatología, Obstetricia, Repr...",Factores de riesgo asociados para el desarroll...,IntroducciónEn los EE. UU. cada año ocurren ap...,,https://www.elsevier.es/es-revista-perinatolog...,https://www.elsevier.es/es-revista-perinatolog...
4,Perinatología y Reproducción Humana,III. Medicina y Ciencias de la Salud,"Perinatología, Neonatología, Obstetricia, Repr...",Resultados maternos y neonatales de trillizos ...,AntecedentesLa incidencia de embarazos múltipl...,,https://www.elsevier.es/es-revista-perinatolog...,https://www.elsevier.es/es-revista-perinatolog...
...,...,...,...,...,...,...,...,...
69,Perinatología y Reproducción Humana,III. Medicina y Ciencias de la Salud,"Perinatología, Neonatología, Obstetricia, Repr...","Células madre, una nueva alternativa médica",IntroducciónLas células madre son células indi...,,https://www.elsevier.es/es-revista-perinatolog...,https://www.elsevier.es/es-revista-perinatolog...
70,Perinatología y Reproducción Humana,III. Medicina y Ciencias de la Salud,"Perinatología, Neonatología, Obstetricia, Repr...",Infecciones por gérmenes atípicos en el recién...,IntroducciónLas infecciones en general represe...,,https://www.elsevier.es/es-revista-perinatolog...,https://www.elsevier.es/es-revista-perinatolog...
71,Perinatología y Reproducción Humana,III. Medicina y Ciencias de la Salud,"Perinatología, Neonatología, Obstetricia, Repr...","Un tributo a Roberto Caldeyro-Barcia, consider...",IntroducciónDurante el III Congreso Latinoamer...,,https://www.elsevier.es/es-revista-perinatolog...,https://www.elsevier.es/es-revista-perinatolog...
72,Perinatología y Reproducción Humana,III. Medicina y Ciencias de la Salud,"Perinatología, Neonatología, Obstetricia, Repr...",Estómago intratorácico en un recién nacido: ¿h...,IntroducciónPresentamos el caso de un paciente...,,https://www.elsevier.es/es-revista-perinatolog...,https://www.elsevier.es/es-revista-perinatolog...


In [68]:
df_catalogo.to_csv('Revista095.02.csv', index=False)

In [69]:
df=df_catalogo[~ df_catalogo["Titulo Articulo"].isin(['Editorial'])]

In [70]:
df.to_csv('Revista095.02.csv', index=False)